In [1]:
import poppler

In [2]:
from pyzerox import zerox
import os
import json
import asyncio

/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [10]:

file_path = "/data/Otherscape/Tokyo-Otherscape Setting Book.pdf" ## local filepath and file URL supported

    
output_dir = "./output_test" ## directory to save the consolidated markdown file
result = await zerox(file_path=file_path, model="gpt-4o-mini", output_dir=output_dir,
                         select_pages=None)
result

ZeroxOutput(completion_time=502016.239, file_name='tokyo_otherscape_setting_book', input_tokens=512577, output_tokens=211115, pages=[Page(content='# TOKYO :OTHERSCAPE\n## JAPAN SETTING BOOK\n\n**Property of City of Mist issued to Tierra Duffield. Order #218284**  \n**Powered by: City of Mist**', content_length=143, page=1), Page(content='# CREDITS\n\n**Creator, Lead Game Designer, and Creative Director:**  \nAmit Moshe\n\n**Game Designers:**  \nEran Aviram, Itamar Karbian, Kelly Black\n\n**Developer:**  \nKelly Black\n\n**Editor:**  \nEran Aviram\n\n**Proofreader:**  \nMatthew Pook\n\n**Cultural Consultant:**  \nDr. Emma Yasui, Isago Fukuda, Dr. Yiftach Raphael Goveren\n\n**Game Producers:**  \nOmer Shapira, Inbar Barzilay\n\n**Art Production:**  \nErina Liddle\n\n**Lead Writers:**  \nColin "Mephit James" Wilson, Kelly Black, Tony Pi\n\n**Writers:**  \nAkemi Maniwa, Amit Moshe, Eran Aviram, Jordan Marshall, Kend\'or Wilson, Kevin Carpenter, Kristoff Alejo, Lexi Antoku, Nathan Blades, D

In [11]:
def with_subheadings():
    import re

    # Initialize the array to store the results
    final_results = []

    # Regular expression to match headings and subheadings
    heading_regex = r'^(#+)\s+(.*)'  # Matches markdown headings like # Heading or ## Subheading

    # Temporary variables to store current heading, subheading, and content
    current_heading = None
    current_subheading = None
    current_content = []
    current_pages = []

    # Iterate over the pages in the results
    for page_number, page in enumerate(result.pages, start=1):
        lines = page.content.splitlines()
        for line in lines:
            match = re.match(heading_regex, line)
            if match:
                # If a new heading or subheading is found, save the previous one
                if current_heading or current_subheading:
                    final_results.append({"heading": current_heading,
                                        "subheading": current_subheading,
                                        "page": current_pages,
                                        "content": '\n'.join(current_content)})
                
                # Update the current heading/subheading
                level, title = match.groups()
                if len(level) == 1:
                    # It's a main heading
                    current_heading = title
                    current_subheading = None  # Reset subheading
                elif len(level) > 1:
                    # It's a subheading
                    current_subheading = title

                # Reset content and pages
                current_content = []
                current_pages = [page_number]
            else:
                # Add line to current content
                current_content.append(line)
                if page_number not in current_pages:
                    current_pages.append(page_number)

    # Add the last heading/subheading to the results if needed
    if current_heading or current_subheading:
        key = f"{current_heading or ''}-{current_subheading or ''}-pages:{','.join(map(str, current_pages))}"
        final_results.append({key: '\n'.join(current_content)})

    # Output final results
    return final_results


In [12]:
def without_subheadings():
    import re

    # Initialize the array to store the results
    final_results = []

    # Regular expression to match headings and subheadings
    heading_regex = r'^(#)\s+(.*)'  # Matches markdown headings1 like # Heading

    # Temporary variables to store current heading, subheading, and content
    current_heading = None
    current_content = []
    current_pages = []

    # Iterate over the pages in the results
    for page_number, page in enumerate(result.pages, start=1):
        lines = page.content.splitlines()
        for line in lines:
            match = re.match(heading_regex, line)
            if match:
                # If a new heading or subheading is found, save the previous one
                if current_heading:
                    final_results.append({"heading": current_heading,
                                        "page": current_pages,
                                        "content": '\n'.join(current_content)})
                
                # Update the current heading/subheading
                _, title = match.groups()
                current_heading = title

                # Reset content and pages
                current_content = []
                current_pages = [page_number]
            else:
                # Add line to current content
                current_content.append(line)
                if page_number not in current_pages:
                    current_pages.append(page_number)

    # Add the last heading/subheading to the results if needed
    if current_heading:
        key = f"{current_heading or ''}-pages:{','.join(map(str, current_pages))}"
        final_results.append({key: '\n'.join(current_content)})

    # Output final results
    return final_results


In [13]:
from utils.elastic import elastic_request
import requests

def ingest(doc, title, index):
    cleaned_title = title.replace("?", "").replace("/", "-")
    rslt = elastic_request(method=requests.put,
                           url=f"{index}/_doc/{cleaned_title}?pipeline=clean_and_embed",
                           data={"content": doc["content"], "pages": doc["page"]})
    return rslt


In [14]:
i = 0
for rslt in without_subheadings():
    if not rslt.get("content"):
        continue
    title = rslt["heading"]    
    resp = ingest(rslt, title, "otherscape--tokyo-otherscape")
    
    try:
        resp.raise_for_status()
    except Exception as e:
        print("Error: ", e)
        print(resp.json())
        raise
    i += 1
    if i % 10 == 0:
        print(rslt)

/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn

{'heading': 'EMERGENCY-POWERED CITY-STATE', 'page': [11], 'content': 'Japan was the first to mass-utilize nanotech, and the first to get burned by it in a disaster that quite literally reshaped the nation. Dubbing the government response inadequate, Tokyo itself took over all aspects of governance and expanded westwards aggressively while becoming the new de facto regime. The Nano-buta (page 25) gained almost limitless jurisdiction that they retain even now, long after the disaster has been contained. This power was quickly siphoned by corrupt officials into unholy covenants with the already-in-league corporate zaibatsus (page 26) and Yakuza crime syndicates (page 27), further subjecting the local population to the unchecked rule of modern-day shoguns and daimyos.\n'}


/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn

{'heading': 'THE NANOTECH DISASTER', 'page': [14], 'content': 'Nanotech is taboo in Tokyo. Unregulated advancements in nanotechnology have resulted in several disasters, and at some point in the recent past Japan suffered a major nanite accident. A research center run by a biotechnology and geo-engineering conglomerate accidentally let loose a powerfully mutagenic and self-replicating nanite swarm, and millions died in the resulting catastrophe.\n\nNanotechnology has since been curtailed and largely outlawed, although the megacorps, who fear losing the race to the competition aboard, are doing their best to change public opinion and regulations.'}


/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn

...

/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn

{'heading': 'ACT I: GHOSTED', 'page': [299], 'content': '\nTo make their meeting, the crew will first need to track down the current location of the Ghost Market (page 159), the ever-changing and ever-moving underground night market of the Megacity.\n\nThis can be achieved with divination, by hacking an invite (such as a nefarious arms dealer), or by leveraging contacts in the Megacity’s criminal underworld. This could be resolved as a quick outcome with a tier-3 opposing status such as **secretive-3** or **firewall-3**, with potentially substantial consequences, such as being **indebted-3** to their contact.\n\nUnless they have had dealings with the Ghost Market before, the crew are unlikely to get more than a one-time invitation, but that’s enough.\n\n## ENTERING THE GHOST MARKET\n\nThe night is hot, noisy, and smells of burnt plastic. So everything’s as usual. The only hint that something’s off, that you’re on the right direction toward the Ghost Market, is the occasional cherry pet

/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn

{'heading': 'AFTERMATH', 'page': [316, 317], 'content': "\nHowever things ended, the consequences should be lasting.\n\n- If the Junjō was successfully sealed away, the Amatsukami take a step back. Clearly humanity was not ready for their gift. Or perhaps some other force intervened? The gods of heaven will send some of their loyal servants to spy on the crew for several months, to see if they are perhaps under some corrupting influence. They might turn to Great King Enma (page 168) and ask him to test the crew. The Chief Inspector, meanwhile, is probably impressed enough with the crew to turn to them again with future delicate matters, and perhaps offer some of them a job in the Bureau. Whenever the crew face injustice and the corruption of mankind, remind them of the choice they made to shun enlightenment.\n\n- If the Junjō was released from the skybridge, tens of thousands in the district will be transformed like the techs in the Maikubi-kai labs, with the entire district being quar

/workspaces/llm-dungeon-master/versions/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
